In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
batch_size = 32
n_epochs = 50
steps_per_epoch = 100

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        horizontal_flip=True,
        rotation_range=10.,
        width_shift_range=0.2,
        height_shift_range=0.2)

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
classes = np.loadtxt('Data/videos/classes.txt') 
train_generator = train_datagen.flow_from_directory(
    'Data/videos/train/',
    target_size=(299, 299),
    batch_size=batch_size,
    classes=classes,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    'Data/videos/test/',
    target_size=(299, 299),
    batch_size=batch_size,
    classes=classes,
    class_mode='categorical')

In [ ]:
inception_model = InceptionV3(include_top=False)

In [ ]:
x = inception_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(classes), activation='softmax')(x)
model = Model(inputs=inception_model.input, outputs=predictions)

In [ ]:
for layer in model.layers[:-2]:
    layer.trainable = False

In [ ]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=10,
    epochs=n_epochs)

In [ ]:
model.layers.pop()
model.layers.pop()
model.outputs = [model.layers[-1].output]
model.output_layers = [model.layers[-1]]
model.layers[-1].outbound_nodes = []

In [ ]:
for video in data:
    path = 'Data/videos/' + str(video)
    frames = get_frame(video)
    sequence = []
    for image in frames:
        features = model.predict(image)
        sequence.append(features)
    np.savetxt(path, sequence)

In [ ]:
model_lstm = Sequential()
model_lstm.add(LSTM(1024, return_sequences=True, input_shape=input_shape, dropout=dropout))
model_lstm.add(Flatten())
model_lstm.add(Dense(512, activation='relu'))
model_lstm.add(Dropout(dropout))
model_lstm.add(Dense(self.nb_classes, activation='softmax'))

In [ ]:
batch_size_lstm = 32
n_epochs_lstm = 40

In [ ]:
model_lstm.fit(X_train,
            y_train,
            batch_size=batch_size_lstm,
            validation_data=(X_val, y_val),
            verbose=1,
            epochs=n_epochs_lstm)